In [1]:
#!/usr/bin/env python3
import os
import sys
import subprocess
import time
import difflib
import re

COMPILER_PATH = "./build/Compiler"  # 编译器路径
TESTCASE_DIR = "testcases"          # 测试用例目录
COMPARATOR = "/usr/bin/riscv64-unknown-elf-gcc" 
QEMU_PATH = "qemu-riscv64"          # RISC-V模拟器
TIMEOUT_DURATION = 2                # 运行超时阈值(秒)
COMPILE_TIMEOUT = 60                # 编译超时阈值(秒)
OUTPUT_DIR = "./output"             # 输出目录（仅用于.bin和.actual文件）

# ===== 初始化统计变量 =====
CE_list = []      # 编译错误
RE_list = []      # 运行时错误
TLE_list = []     # 超时错误
WA_list = []      # 错误答案
AC_list = []      # 通过用例
Time_Out = []     # 超时函数
Bad_test = []     # 无效测试
total_time = 0    # 总耗时

# ===== 准备输出目录 =====
os.makedirs(OUTPUT_DIR, exist_ok=True)  # 仅用于.bin和.actual文件

# ===== 收集测试用例 =====
test_list = []
for root, dirs, files in os.walk(TESTCASE_DIR):
    for file in files:
        if file.endswith(".sy"):
            test_list.append(os.path.join(root, file))

# ===== 主测试循环 =====
for test in test_list:
    base_name = os.path.basename(test).replace(".sy", "")
    base_path = os.path.dirname(test)
    out_file = os.path.join(base_path, base_name + ".out")
    in_file = os.path.join(base_path, base_name + ".in")
    # .s文件生成在.sy文件同目录下
    s_file = os.path.join(base_path, base_name + ".s")
    bin_file = os.path.join(OUTPUT_DIR, base_name + ".bin")
    actual_out = os.path.join(OUTPUT_DIR, base_name + ".actual")
    
    print(f"Processing: {test}")

    # 检查.out文件是否存在
    if not os.path.exists(out_file):
        Bad_test.append(test)
        print(f"  Bad Test: Missing .out file for {test}")
        continue

    # === 步骤1: 编译源文件生成汇编 ===
    try:
        compile_args = [COMPILER_PATH, test, "-o", s_file]
        # 添加--march选项传递给编译器（如果需要）
        # compile_args += ["--march=rv64gc"]
        subprocess.run(compile_args, timeout=COMPILE_TIMEOUT, check=True,
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except subprocess.TimeoutExpired:
        CE_list.append(test)
        Time_Out.append(test)
        print(f"  Compiler Timeout: {test}")
        continue
    except subprocess.CalledProcessError as e:
        CE_list.append(test)
        print(f"  Compiler Error: {test}\n    {e.stderr.decode().strip()}")
        continue
    
    # === 步骤2: 编译汇编生成可执行文件（使用图片中的参数）===
    try:
        # 核心修改：添加rv64gc选项（G=IMAFFD，C=压缩指令）
        assemble_args = [
            COMPARATOR, 
            "-static", 
            "-march=rv64gc",  # 使用图片中的精确参数
            "-mabi=lp64",     # 64位ABI
            s_file, 
            "-o", 
            bin_file,
            "-lm"             # 链接数学库（如果有浮点运算）
        ]
        subprocess.run(assemble_args, timeout=TIMEOUT_DURATION, check=True,
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    except subprocess.TimeoutExpired:
        CE_list.append(test)
        Time_Out.append(test)
        print(f"  Assembler Timeout: {test}")
        continue
    except subprocess.CalledProcessError as e:
        # 详细输出汇编器错误信息
        error_msg = e.stderr.decode().strip()
        CE_list.append(test)
        print(f"  Assembler Error: {test}\n    {error_msg}")
        
        # 保存汇编错误日志以便调试
        asm_error_log = os.path.join(base_path, base_name + "_asm_error.log")
        with open(asm_error_log, "w") as err_file:
            err_file.write(f"Assembler command: {' '.join(assemble_args)}\n")
            err_file.write(f"Error details:\n{error_msg}\n")
        continue
    except FileNotFoundError:
        CE_list.append(test)
        print(f"  Error: Assembler not found at {COMPARATOR}")
        continue
    
    # === 步骤3: 执行测试并计时 ===
    start_time = time.time()
    try:
        # 准备输入数据
        input_data = None
        if os.path.exists(in_file):
            with open(in_file, "rb") as f:
                input_data = f.read()
        
        # 执行程序（指定64位架构）
        run_args = [QEMU_PATH, "-cpu", "rv64", bin_file]
        result = subprocess.run(run_args, input=input_data, timeout=TIMEOUT_DURATION,
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        elapsed = time.time() - start_time
        total_time += elapsed
        
        # 处理输出
        output = result.stdout.decode(errors="ignore")
        if not output.endswith('\n'):
            output += '\n'
        output += str(result.returncode) + '\n'
        
        with open(actual_out, "w") as f:
            f.write(output)
    
    except subprocess.TimeoutExpired:
        TLE_list.append(test)
        Time_Out.append(test)
        total_time += TIMEOUT_DURATION
        print(f"  Timeout Error: {test}")
        continue
    except Exception as e:
        # 保存运行错误日志
        runtime_error_log = os.path.join(OUTPUT_DIR, base_name + "_runtime_error.log")
        with open(runtime_error_log, "w") as err_file:
            err_file.write(f"Runtime error: {str(e)}\n")
            if result:
                err_file.write(f"Stdout: {result.stdout.decode(errors='ignore')}\n")
                err_file.write(f"Stderr: {result.stderr.decode(errors='ignore')}\n")
                
        RE_list.append(test)
        print(f"  Runtime Error: {test}\n    {str(e)}")
        continue
    
    # === 步骤4: 兼容性更强的输出比较 ===
    try:
        with open(out_file, "r") as f:
            expected_output = f.read()
        
        # 规范化比较（允许±1误差和浮点容错）
        def normalize_output(text):
            lines = text.strip().splitlines()
            # 尝试将数值解析为浮点数，否则保持原字符串
            normalized = []
            for line in lines:
                try:
                    # 浮点数容差处理
                    val = float(line)
                    normalized.append(f"{val:.6f}")  # 限制精度
                except ValueError:
                    # 处理整数/字符串
                    normalized.append(line.strip())
            return normalized
        
        norm_output = normalize_output(output)
        norm_expected = normalize_output(expected_output)
        
        # 容错比较
        if len(norm_output) != len(norm_expected):
            WA_list.append(test)
            print(f"  Wrong Answer: Output line count mismatch")
        else:
            all_match = True
            for i, (out_line, exp_line) in enumerate(zip(norm_output, norm_expected)):
                if out_line != exp_line:
                    try:
                        # 浮点数容差比较
                        out_val = float(out_line)
                        exp_val = float(exp_line)
                        if abs(out_val - exp_val) > 1e-5:  # 万分之一误差容忍
                            all_match = False
                    except ValueError:
                        all_match = False
            
            if all_match:
                AC_list.append(test)
                print(f"  Accepted: {test} ({elapsed:.3f}s)")
            else:
                WA_list.append(test)
                print(f"  Wrong Answer: {test}")
                
                # 生成详细差异报告
                diff = difflib.unified_diff(
                    output.splitlines(), expected_output.splitlines(),
                    fromfile="Actual", tofile="Expected", lineterm=''
                )
                diff_report = os.path.join(OUTPUT_DIR, base_name + ".diff")
                with open(diff_report, "w") as df:
                    df.write("\n".join(diff))
    except Exception as e:
        WA_list.append(test)
        print(f"  Output Comparison Failed: {test}\n    {str(e)}")

# ===== 打印统计结果 =====
print("\n===== Test Summary =====")
print(f"Compiler Error: Total: {len(CE_list)}")
print(f"Runtime Error: Total: {len(RE_list)}")
print(f"Timeout Error: Total: {len(TLE_list)}")
print(f"Wrong Answer: Total: {len(WA_list)}")
print(f"TimeOut Function: {len(Time_Out)}")
print(f"Bad Test: Total: {len(Bad_test)}")
print(f"Accepted: Total: {len(AC_list)}")
print(f"TOTAL TIME: {total_time:.3f} seconds")
print("========================")

# 保存详细报告
with open(os.path.join(OUTPUT_DIR, "test_report.log"), "w") as report:
    report.write("===== Detailed Test Report =====\n")
    report.write(f"Total Test Cases: {len(test_list)}\n\n")
    report.write(f"Compiler: {COMPILER_PATH}\n")
    report.write(f"Assembler: {COMPARATOR}\n")
    report.write(f"QEMU: {QEMU_PATH}\n")
    report.write(f"Compile Options: -static -march=rv64gc -mabi=lp64\n\n")
    
    if CE_list:
        report.write("\n===== Compiler Errors =====\n")
        for test in CE_list:
            report.write(f"  {test}\n")
    
    if RE_list:
        report.write("\n===== Runtime Errors =====\n")
        for test in RE_list:
            report.write(f"  {test}\n")
    
    if TLE_list:
        report.write("\n===== Timeout Errors =====\n")
        for test in TLE_list:
            report.write(f"  {test}\n")
    
    if WA_list:
        report.write("\n===== Wrong Answers =====\n")
        for test in WA_list:
            report.write(f"  {test}\n")
    
    if Bad_test:
        report.write("\n===== Bad Tests (missing .out) =====\n")
        for test in Bad_test:
            report.write(f"  {test}\n")
    
    if AC_list:
        report.write("\n===== Accepted Tests =====\n")
        for test in AC_list:
            report.write(f"  {test}\n")
    
    report.write(f"\n===== Performance Summary =====\n")
    report.write(f"Total Execution Time: {total_time:.3f} seconds\n")
    if test_list:
        avg_time = total_time / len(AC_list) if AC_list else 0
        report.write(f"Average Time per Test: {avg_time:.4f} seconds\n")

print(f"\nDetailed report saved to: {os.path.join(OUTPUT_DIR, 'test_report.log')}")

Processing: testcases/h_functional/05_param_name.sy
  Compiler Error: testcases/h_functional/05_param_name.sy
    
Processing: testcases/h_functional/39_fp_params.sy
  Compiler Error: testcases/h_functional/39_fp_params.sy
    
Processing: testcases/h_functional/24_array_only.sy
  Compiler Error: testcases/h_functional/24_array_only.sy
    
Processing: testcases/h_functional/06_func_name.sy
  Assembler Error: testcases/h_functional/06_func_name.sy
    /usr/lib/gcc/riscv64-unknown-elf/13.2.0/../../../riscv64-unknown-elf/bin/ld: cannot find crt0.o: No such file or directory
/usr/lib/gcc/riscv64-unknown-elf/13.2.0/../../../riscv64-unknown-elf/bin/ld: cannot find -lm: No such file or directory
/usr/lib/gcc/riscv64-unknown-elf/13.2.0/../../../riscv64-unknown-elf/bin/ld: cannot find -lc: No such file or directory
/usr/lib/gcc/riscv64-unknown-elf/13.2.0/../../../riscv64-unknown-elf/bin/ld: cannot find -lgloss: No such file or directory
collect2: error: ld returned 1 exit status
Processing: te